In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import re

In [ ]:
train_df = pd.read_json('../input/train.json')
test_df = pd.read_json('../input/test.json')

In [ ]:
train_df.head()

In [ ]:
print('Training size : {}\nTest size : {}'.format(train_df.shape[0],test_df.shape[0]))

In [ ]:
print('Number of unique cuisines ' + str(len(train_df['cuisine'].value_counts())))

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
cuisine_name = train_df['cuisine'].value_counts().index
cuisine_count = train_df['cuisine'].value_counts().values
plt.figure(figsize=(20,5))
sns.barplot(cuisine_name, cuisine_count)
plt.xlabel('Cuisine')
plt.ylabel('#Occurrences')
plt.title('Most common cuisines bar plot.')
plt.show()

In [ ]:
# bar plot of most common ingredients
ingredients = list()
for ings in train_df['ingredients']:
    for ing in ings:
        ingredients.append(ing)

In [ ]:
print('Total number of ingredients (including duplicates): ' + str(len(ingredients)))

In [ ]:
from collections import Counter
ingredients_counter = Counter(ingredients)
k = 15
plt.figure(figsize=(20,5))
top_k_ingredients = ingredients_counter.most_common(k)
name, counts = zip(*top_k_ingredients)
sns.barplot(list(name), list(counts))
plt.show()

In [ ]:
train_df['ingredients'] = train_df['ingredients'].map(lambda x: " ".join(x))
test_df['ingredients'] = test_df['ingredients'].map(lambda x: " ".join(x))

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer

In [ ]:
tfidf = TfidfVectorizer()
train_features = tfidf.fit_transform(train_df['ingredients'])
test_features = tfidf.transform(test_df['ingredients'])

In [ ]:
count_vectorizer = CountVectorizer()
train_features_cv = count_vectorizer.fit_transform(train_df['ingredients'])
test_features_cv = count_vectorizer.transform(test_df['ingredients'])

In [ ]:
label_encoder = LabelEncoder()
train_labels = label_encoder.fit_transform(train_df['cuisine'])

In [ ]:
X = pd.DataFrame(train_features.todense())

In [ ]:
X_cv = pd.DataFrame(train_features_cv.todense())

In [ ]:
X_tfidf_cv = pd.concat([X,X_cv],axis=1)

In [ ]:
y = train_labels

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)

In [ ]:
X_train_cv, X_test_cv, y_train, y_test = train_test_split(X_cv, y, random_state=42, test_size=0.2)

In [ ]:
X_train_tfidf_cv, X_test_tfidf_cv, y_train, y_test = train_test_split(X_cv, y, random_state=42, test_size=0.2)

In [ ]:
# TF-IDF features
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(n_estimators=20)
rfc.fit(X_train, y_train)
rfc.score(X_test, y_test)

In [ ]:
# Count Vectorizer features
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier(n_estimators=30)
# rfc.fit(X_train_cv, y_train)
# rfc.score(X_test_cv, y_test)

In [ ]:
# Count Vectorizer and TF-IDF combined features
# from sklearn.ensemble import RandomForestClassifier
# rfc = RandomForestClassifier(n_estimators=30)
# rfc.fit(X_train_tfidf_cv, y_train)
# rfc.score(X_test_tfidf_cv, y_test)

In [ ]:
test_X = pd.DataFrame(test_features.todense())

In [ ]:
test_X_cv = pd.concat([pd.DataFrame(test_features.todense()),pd.DataFrame(test_features_cv.todense())],axis=1)

In [ ]:
predict_test = rfc.predict(test_X)

In [ ]:
predicted_cuisines = label_encoder.inverse_transform(predict_test)

In [ ]:
pd.DataFrame({'id':test_df['id'], 'cuisine':predicted_cuisines}).to_csv('submission.csv', index=False)

In [ ]:
predicted_cuisines[:10]

In [ ]:
predicted = rfc.predict(X_train)

In [ ]:
np.mean(y_train==predicted)

In [ ]:
len(predicted_cuisines)